In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
df=pd.read_csv('./datasets/final/release_train_patients.csv')
pd.set_option('display.max_colwidth', None)

In [3]:
df.head()

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,18,"[['Bronchitis', 0.19171203430383882], ['Pneumonia', 0.17579340398940366], ['URTI', 0.1607809719801254], ['Bronchiectasis', 0.12429044460990353], ['Tuberculosis', 0.11367177304035844], ['Influenza', 0.11057936110639896], ['HIV (initial infection)', 0.07333003867293564], ['Chagas', 0.04984197229703562]]",M,URTI,"['E_48', 'E_50', 'E_53', 'E_54_@_V_161', 'E_54_@_V_183', 'E_55_@_V_89', 'E_55_@_V_108', 'E_55_@_V_167', 'E_56_@_4', 'E_57_@_V_123', 'E_58_@_3', 'E_59_@_3', 'E_77', 'E_79', 'E_91', 'E_97', 'E_201', 'E_204_@_V_10', 'E_222']",E_91
1,21,"[['HIV (initial infection)', 0.5189500564407601], ['Chagas', 0.3217819010436332], ['Scombroid food poisoning', 0.13496758062695968], ['Sarcoidosis', 0.024300461888647054]]",M,HIV (initial infection),"['E_9', 'E_27', 'E_50', 'E_51', 'E_53', 'E_54_@_V_198', 'E_55_@_V_62', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_7', 'E_57_@_V_123', 'E_58_@_7', 'E_59_@_2', 'E_91', 'E_115', 'E_129', 'E_130_@_V_138', 'E_131_@_V_10', 'E_132_@_0', 'E_133_@_V_90', 'E_133_@_V_91', 'E_133_@_V_95', 'E_133_@_V_110', 'E_133_@_V_111', 'E_134_@_6', 'E_135_@_V_12', 'E_136_@_0', 'E_148', 'E_162', 'E_189', 'E_204_@_V_10']",E_50
2,19,"[['Bronchitis', 0.11278064619119596], ['Pneumonia', 0.10048134562119852], ['Unstable angina', 0.08462979865697953], ['Possible NSTEMI / STEMI', 0.08343170881871906], ['Bronchiectasis', 0.07764173703530038], ['Boerhaave', 0.07708363099826726], ['Spontaneous rib fracture', 0.07682349455456361], ['Pericarditis', 0.07504681799224715], ['URTI', 0.06705952510476514], ['Stable angina', 0.06667817325012103], ['GERD', 0.06455232704351052], ['Scombroid food poisoning', 0.058790058668404095], ['Chagas', 0.02923384001251551], ['Sarcoidosis', 0.016406922262218876], ['Panic attack', 0.009359973789993502]]",F,Pneumonia,"['E_53', 'E_54_@_V_179', 'E_54_@_V_192', 'E_55_@_V_29', 'E_55_@_V_55', 'E_55_@_V_56', 'E_55_@_V_170', 'E_55_@_V_171', 'E_56_@_5', 'E_57_@_V_123', 'E_58_@_2', 'E_59_@_6', 'E_77', 'E_78', 'E_79', 'E_94', 'E_106', 'E_118', 'E_123', 'E_124', 'E_129', 'E_130_@_V_156', 'E_131_@_V_12', 'E_132_@_4', 'E_133_@_V_54', 'E_133_@_V_85', 'E_134_@_0', 'E_135_@_V_10', 'E_136_@_0', 'E_144', 'E_175', 'E_181', 'E_201', 'E_204_@_V_10']",E_77
3,34,"[['URTI', 0.23859396799565236], ['Cluster headache', 0.18566159722723558], ['Bronchitis', 0.1773471504190298], ['Chronic rhinosinusitis', 0.14960995823729417], ['Acute rhinosinusitis', 0.10748772366243657], ['Chagas', 0.07064980122917579], ['Anemia', 0.07064980122917579]]",F,URTI,"['E_48', 'E_53', 'E_54_@_V_183', 'E_55_@_V_89', 'E_55_@_V_109', 'E_55_@_V_124', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_7', 'E_57_@_V_123', 'E_58_@_5', 'E_59_@_4', 'E_97', 'E_181', 'E_201', 'E_204_@_V_10']",E_53
4,36,"[['URTI', 0.23677812769175735], ['Influenza', 0.1842006792554458], ['Bronchitis', 0.1651209569301098], ['Cluster headache', 0.1430457852409739], ['Chronic rhinosinusitis', 0.139295948156182], ['Chagas', 0.06577925136276556], ['Anemia', 0.06577925136276556]]",M,URTI,"['E_49', 'E_50', 'E_53', 'E_54_@_V_183', 'E_55_@_V_62', 'E_55_@_V_124', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_5', 'E_57_@_V_123', 'E_58_@_4', 'E_59_@_2', 'E_97', 'E_144', 'E_181', 'E_201', 'E_204_@_V_10']",E_201


In [4]:
selected_columns = ["PATHOLOGY", "EVIDENCES"]
new_df = df[selected_columns]

# Write the new DataFrame to a new CSV file
# new_df.to_csv("new.csv", index=False)

In [7]:
new_df.head()

,PATHOLOGY,EVIDENCES
0,URTI,"['E_48', 'E_50', 'E_53', 'E_54_@_V_161', 'E_54_@_V_183', 'E_55_@_V_89', 'E_55_@_V_108', 'E_55_@_V_167', 'E_56_@_4', 'E_57_@_V_123', 'E_58_@_3', 'E_59_@_3', 'E_77', 'E_79', 'E_91', 'E_97', 'E_201', 'E_204_@_V_10', 'E_222']"
1,HIV (initial infection),"['E_9', 'E_27', 'E_50', 'E_51', 'E_53', 'E_54_@_V_198', 'E_55_@_V_62', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_7', 'E_57_@_V_123', 'E_58_@_7', 'E_59_@_2', 'E_91', 'E_115', 'E_129', 'E_130_@_V_138', 'E_131_@_V_10', 'E_132_@_0', 'E_133_@_V_90', 'E_133_@_V_91', 'E_133_@_V_95', 'E_133_@_V_110', 'E_133_@_V_111', 'E_134_@_6', 'E_135_@_V_12', 'E_136_@_0', 'E_148', 'E_162', 'E_189', 'E_204_@_V_10']"
2,Pneumonia,"['E_53', 'E_54_@_V_179', 'E_54_@_V_192', 'E_55_@_V_29', 'E_55_@_V_55', 'E_55_@_V_56', 'E_55_@_V_170', 'E_55_@_V_171', 'E_56_@_5', 'E_57_@_V_123', 'E_58_@_2', 'E_59_@_6', 'E_77', 'E_78', 'E_79', 'E_94', 'E_106', 'E_118', 'E_123', 'E_124', 'E_129', 'E_130_@_V_156', 'E_131_@_V_12', 'E_132_@_4', 'E_133_@_V_54', 'E_133_@_V_85', 'E_134_@_0', 'E_135_@_V_10', 'E_136_@_0', 'E_144', 'E_175', 'E_181', 'E_201', 'E_204_@_V_10']"
3,URTI,"['E_48', 'E_53', 'E_54_@_V_183', 'E_55_@_V_89', 'E_55_@_V_109', 'E_55_@_V_124', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_7', 'E_57_@_V_123', 'E_58_@_5', 'E_59_@_4', 'E_97', 'E_181', 'E_201', 'E_204_@_V_10']"
4,URTI,"['E_49', 'E_50', 'E_53', 'E_54_@_V_183', 'E_55_@_V_62', 'E_55_@_V_124', 'E_55_@_V_166', 'E_55_@_V_167', 'E_56_@_5', 'E_57_@_V_123', 'E_58_@_4', 'E_59_@_2', 'E_97', 'E_144', 'E_181', 'E_201', 'E_204_@_V_10']"


In [6]:
X = new_df.drop(columns=['PATHOLOGY'])
y = new_df['PATHOLOGY']

# Create train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline
model = make_pipeline(
    TfidfVectorizer(),
    LogisticRegression(random_state=42, solver='lbfgs', max_iter=200)
)


In [7]:
# Train the model
model.fit(X['EVIDENCES'], y)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=200, random_state=42))])

In [10]:
df_test=pd.read_csv('./datasets/final/release_test_patients.csv')
pd.set_option('display.max_colwidth', None)

new_df_test = df_test[selected_columns]

In [11]:
from pathlib import Path
Path().cwd().parent.parent

PosixPath('/Volumes/Arham/A2/BMS College stufff/PBL/Final Year Project - HopSlot/Dev')

In [12]:
import joblib

joblib.__version__

'1.4.2'

In [13]:
import pandas as pd
import joblib, dill
from pathlib import Path
pwd = Path.cwd()
path = Path(pwd.parent / "HopSlot-FullStack/django-api/predict_symptoms/trained_models")

joblib.dump(model, path / 'model.pkl')


['/Volumes/Arham/A2/BMS College stufff/PBL/Final Year Project - HopSlot/Dev/Code/HopSlot-FullStack/django-api/predict_symptoms/trained_models/model.pkl']

In [8]:
def transformAPIRequest(appointment):
    evidences = []

    selected_columns = ["question_en", "name", "value_meaning", "data_type"]
    df = pd.read_json("datasets/final/release_evidences.json").T
    df = df[selected_columns]
    data_dict = df.to_dict(orient="records")

    for symptom in appointment["symptoms"]:
        search_key = symptom["name"]
        res = next(item for item in data_dict if search_key in item["question_en"])
        if res["data_type"] == "B":
            evidences.append(res["name"])
        elif res["data_type"] == "M":
            value_meaning = symptom["values"]
            evidences.append(res["name"] + "_@_" + value_meaning[0])
        else:
            value_meaning = symptom["values"]
            evidences.append(res["name"] + "_@_" + value_meaning)

    return evidences  # Your model inputs which will be used for prediction


def mapPathologyToSeverity(pathology):
    df = pd.read_json("datasets/final/release_conditions.json").T
    selected_columns = ["condition_name", "severity"]
    df = df[selected_columns]
    data_dict = df.to_dict(orient="records")
    return next(item for item in data_dict if item["condition_name"] == pathology)[
        "severity"
    ]


In [12]:
appointmentBatch = [
  {
    "appointmentId": "12233-asdas-21232",
    "symptoms": [{
      "name": "fever",
      "description": "fever for 2 days",
      "type": "B"
    }]
  },
  {
    "appointmentId": "34232asdas-asda-2123",
    "symptoms": [{
      "name": "fever",
      "description": "fever for 1 days",
      "type": "B"
      
    }]
  },
  {
    "appointmentId": "64232asdas-asda-2123",
    "symptoms": [{
      "name": "fatigue",
      "description": "last night fatigue",
      "type": "B",
    }]
  },
   {
    "appointmentId": "74232asdas-asda-2123",
    "symptoms": [{
      "name": "suffocating",
      "description": "suffocating last night",
      "type": "B",
    }]
  },
   {
    "appointmentId": "84232asdas-asda-2123",
    "symptoms": [{
      "name": "diarrhea",
      "description": "dairrhea",
      "type": "B",
    }]
  },
   {
    "appointmentId": "04232asdas-asda-2123",
    "symptoms": [{
      "name": "lightheaded",
      "description": "from morning unconsciousness",
      "type": "B",
    }]
  },
   {
    "appointmentId": "14232asdas-asda-2123",
    "symptoms": [{
      "name": "rash",
      "description": "rash on hand",
      "type": "C",
      "values": "pink"
    }]
  },
   {
    "appointmentId": "24232asdas-asda-2123",
    "symptoms": [{
      "name": "pain",
      "description": "back pain",
      "type": "M",
      "values": "side of the chest(R)"
    }]
  }
]

In [13]:

predictions = []
for appointment in appointmentBatch:
    transformedSymptoms = transformAPIRequest(appointment)
    pathology = model.predict(transformedSymptoms)
    severity = mapPathologyToSeverity(pathology)
    predictions.append(
            {"appointmentId": appointment.get("appointmentId"), "severity": severity}
    )
print(predictions)

[{'appointmentId': '12233-asdas-21232', 'severity': 4}, {'appointmentId': '34232asdas-asda-2123', 'severity': 4}, {'appointmentId': '64232asdas-asda-2123', 'severity': 3}, {'appointmentId': '74232asdas-asda-2123', 'severity': 5}, {'appointmentId': '84232asdas-asda-2123', 'severity': 1}, {'appointmentId': '04232asdas-asda-2123', 'severity': 2}, {'appointmentId': '14232asdas-asda-2123', 'severity': 3}, {'appointmentId': '24232asdas-asda-2123', 'severity': 2}]


In [14]:
y_pred = model.predict(new_df_test['EVIDENCES'])

# Print classification report
print(classification_report(new_df_test['PATHOLOGY'], y_pred))

                                          precision    recall  f1-score   support

     Acute COPD exacerbation / infection       1.00      1.00      1.00      2153
                Acute dystonic reactions       1.00      1.00      1.00      3302
                        Acute laryngitis       1.00      0.99      0.99      3217
                      Acute otitis media       1.00      1.00      1.00      3516
                   Acute pulmonary edema       1.00      1.00      1.00      2598
                    Acute rhinosinusitis       0.97      0.75      0.85      1829
                      Allergic sinusitis       1.00      1.00      1.00      2411
                             Anaphylaxis       1.00      1.00      1.00      3799
                                  Anemia       1.00      1.00      1.00      6842
                     Atrial fibrillation       1.00      1.00      1.00      2831
                               Boerhaave       1.00      1.00      1.00      2083
               